<h1>Survey Neighborhoods in Tokyo for Business Opportunities:Part 1</h1>
Sectioning Airbnb Neighborhoods in Tokyo: Survey the Surroundings for Business Venue<br>
Analysing FourSquare categories  : Find out a prospective Plan.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
0.  <a href="#item0">Introduction: Business Idea and Tools</a>

1.  <a href="#item1">Tokyo AirBnB data : section and survey</a>

2.  <a href="#item2">Scrape and wrangle the Population Dataset from wikipagee and merge</a>

3.  <a href="#item3">Explore the Neighborhoods: Olympic Statdium and business consideration.</a>
    
4.  <a href="#item4">Summary</a>  
    </font>
    </div>

## 0.  <a id="item0">Introduction: Business Idea and Tools</a>

<h2>Project Idea: Key issues</h2>

<ul>
    <li>1: Due to COVID-spread, Tokyo Olympic 2020 is postponed, and the new date is set at the end of July 2021. A huge number of tourists is expected to be flooded in the airbnb hotels, shooping streets, restaurants, bars.</li>
    <li>2: Population of greater Tokyo is close to 40 Milions (To be precise, 38.14 Millions as of today), and it creates a potential cosumer market. Any survey on restaurant/food/bevrage business deserves merits for a study.
 </ul>
 The above Key issues set forth the mission of the project with the following project works.
 <ul>
    <li>1: Sectioning Airbnb Neighborhoods in Tokyo: Survey the Surroundings around Airbnb hotels.</li>
    <li>2: Count and compare the number of srestaurant baed on cusine and categories</li>
    <li>3: Find a potential location for investment on Restaurant Business with specific set of cuisines.<br>
</ul>

<h3>Getting Started</h3>
Foursquare API provides a range of tools for developers to incorporate the up-to-date location data to enhance their projects.
To start using Foursquare, a free acount provides a limited set of tools, which is resonably alright to proof my concept for the course assignment. 
To start coding, first and foremost, import some necessary libraries: pandas.json(to tranforming json file into a pandas dataframe library), folium (plotting library), geopy.geocoders (module to convert an address into latitude and longitude values), and requests (library to handle http requests).

In [1]:
# Import libraries and packages

#library list for airbnb data analysis
from pathlib import Path
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import json
import geojson
import folium
from folium import plugins
from folium.plugins import FastMarkerCluster
from folium.plugins import TimestampedGeoJson
import datetime
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import geoplot as gplt

#libraries for webpage scraping
import requests
from bs4 import BeautifulSoup

#libraries to convert address to latitude and longitudes
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas import json_normalize

%matplotlib inline
#!python --version #Python 3.7.8

## 1.  <a id="item1">Tokyo AirBnB data : section and survey</a>

The open dataset Inside Airbnb databases allows grabbing only the necessary data for the analysis.

In [2]:
# Import listings data, load as pandas dataframe
listings_data = pd.read_csv('listings_june2020.csv')
# Import reviews data, load as a pandas dataframe
reviews_data = pd.read_csv('reviews_june2020.csv')
# Load the geojson file required for mapping as a geodataframe
tokyo_geo = gpd.GeoDataFrame.from_file('neighbourhoods_june2020.geojson')

<h2>data Preparation</h2>

Aggregate the data to find out the number of listings, average price, and latitude/longitude coordinate of each neighborhood. Next, in order for a more quantitative analysis, BeautifulSoup scraping tools is applied on the Special Wards of Tokyo Wikipedia page containing population and area data of the city’s administrative 23 wards.

In [3]:
#listings_data.columns
listings=listings_data[['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365']]
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,35303,"La Casa Gaienmae C Harajuku, Omotesando is nearby",151977,Miyuki,NaN,Shibuya Ku,35.67152,139.71203,Private room,4163,28,18,2018-07-28,0.17,3,89
1,197677,Oshiage Holiday Apartment,964081,Yoshimi & Marek,NaN,Sumida Ku,35.71721,139.82596,Entire home/apt,10993,3,165,2020-03-04,1.55,1,300
2,289597,Private apt in central Tokyo #203,341577,Hide&Kei,NaN,Nerima Ku,35.74267,139.65810,Entire home/apt,4163,28,113,2020-02-17,1.16,2,165
3,370759,"Cozy flat #203, local area YET 10 mins to shib...",1573631,"Gilles,Mayumi,Taiki",NaN,Setagaya Ku,35.66443,139.65707,Entire home/apt,7044,28,103,2020-04-16,1.35,3,361
4,700253,Private apt in central Tokyo #201,341577,Hide&Kei,NaN,Nerima Ku,35.74264,139.65832,Entire home/apt,3949,28,104,2020-05-12,1.11,2,353


## 2. <a id="item2">Scrape and wrangle the Population Dataset from wikipage and merge</a>

BeautifulSoup scraping tool is applied on the Special Wards of Tokyo Wikipedia page containing population and area data of the city’s administrative 23 wards.

In [13]:
response_obj = requests.get('https://en.wikipedia.org/wiki/Special_wards_of_Tokyo').text
soup = BeautifulSoup(response_obj,'lxml')
tokyo_table = soup.find('table', {'class':'wikitable sortable'})

In [14]:
df = pd.DataFrame(columns=['No.', 'Flag', 'Neighborhood', 'Kanji',
                           'Population', 'Density', 'Area_km2', 'Major'])

for row in tokyo_table.findAll("tr"):
    rows = []
    for cell in row.find_all('td'):
        rows.append(cell.text.lstrip('0').replace('\n',''))
    if len(rows) == 8:
        df.loc[len(df)] = rows
#df.head()

In [15]:
df['Population'] = df['Population'].str.replace(',','')
df['Area_km2'] = df['Area_km2'].astype(float)
df = df.sort_values(by=['Neighborhood'])[['Neighborhood','Population','Area_km2']].reset_index(drop=True)

In [16]:
# Merge listings, price 
tokyo_merged_df = tokyo_geo.merge(df, left_index=True, right_index=True)
tokyo_merged_df.head()

,neighbourhood,geometry,listings,price,Neighborhood,Population,Area_km2
0,Adachi Ku,"MULTIPOLYGON (((139.75560 35.81273, 139.76120 ...",193,8981.761658,Adachi,674067,53.25
1,Arakawa Ku,"MULTIPOLYGON (((139.81450 35.73873, 139.80980 ...",381,30647.795276,Arakawa,213648,10.16
2,Bunkyo Ku,"MULTIPOLYGON (((139.76109 35.73213, 139.76241 ...",153,24607.915033,Bunkyō,223389,11.29
3,Chiyoda Ku,"MULTIPOLYGON (((139.77010 35.70526, 139.77310 ...",260,70955.011538,Chiyoda,59441,11.66
4,Chuo Ku,"MULTIPOLYGON (((139.78889 35.69466, 139.79040 ...",598,40699.277592,Chūō,147620,10.21


## 3. <a id="item3">Explore the Neighborhoods: Olympic Statdium and business consideration.</a>

#### FOUR factors co consider: number of listings, average price, distance from olympic venue, and high population


### Define Foursquare Credentials and Version
(Made this cell hidden while on on Github!)

In [18]:
# The code was removed by Watson Studio for sharing.

In [19]:
#Address of Shimokitazawa-station in Setagaya City
address = '2-12 Kitazawa, Setagaya, Tokyo, Japan'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

35.6634677 139.668154


#### Now, let's assume that it is lunch time, and you are craving for italian or Italian food. So, let's define a query to search for food of your interest within 1000 metres from Shimokitaza station.

#### Define URL

In [20]:
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
'There are {} around restaurants Shimokitazawa station.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around restaurants Shimokitazawa station.'

#### Get relevant part of JSON and transform it into a _pandas_ dataframe

In [21]:
# assign relevant part of JSON to venues
items = results['response']['groups'][0]['items']
# tranform venues into a dataframe
dataframe = json_normalize(items) # flatten JSON

## 4. <a id="item4">Summary</a>

This assignment focused on the data preparation for the final assignment of the lecture series.<br>

Based on AirBnB data analysis, the project will suggest for potential shopping streets that are close to Tokyo National Stadium, the main venue for Olympic2020. The considerations are, first and foremost, the average airBnB price is affordable while the location is only a few kilometers away from the main venue of Tokyo Olympic. Second, the area should have high population, which will support the continual growth of any new business. Hence, the business opportunity can be made scalable to sustain the growth after the olympic is over.<br>

Meanwhile, the FourSquare datasets provided key analyses on the specific business. Due to COVID-19 spread and restricted human mobility, neighboring countries of Japan will be comprising the most of the visitors. keeping the scenario in mind and segmenting the food categoires into respective recepies, the popular cuisines will be itimized, and proposed for promising business investment. 

<br> Thank you very much.